In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols 
import statsmodels.stats.api as stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [2]:
df = pd.read_csv(r'C:/Users/Andrei.Baidurov/Marketing_Proyecto/data/marketingcampaigns_clean.csv')

In [3]:
df.head(5)

,campaign_name,start_date,end_date,budget,roi,type,target_audience,channel,conversion_rate,revenue,campaign_duration,net_profit
0,Public-key multi-tasking throughput,2023-04-01 00:00:00,2024-02-23,8082.30,8680%,email,B2B,organic,0.40,709593.48,328,701511.18
1,De-engineered analyzing task-force,2023-02-15 00:00:00,2024-04-22,17712.98,2817%,email,B2C,promotion,0.66,516609.10,432,498896.12
2,Balanced solution-oriented Local Area Network,2022-12-20 00:00:00,2023-10-11,84643.10,441%,podcast,B2B,paid,0.28,458227.42,295,373584.32
3,Distributed real-time methodology,2022-09-26 00:00:00,2023-09-27,14589.75,517%,webinar,B2B,organic,0.19,89958.73,366,75368.98
4,Front-line executive infrastructure,2023-07-07 00:00:00,2024-05-15,39291.90,21%,social media,B2B,promotion,0.81,47511.35,313,8219.45


In [4]:
df['net_profit'] = df['revenue'] - df['budget']

# Display the first few rows to verify the new column
df[['campaign_name', 'revenue', 'budget', 'net_profit']].head()

,campaign_name,revenue,budget,net_profit
0,Public-key multi-tasking throughput,709593.48,8082.30,701511.18
1,De-engineered analyzing task-force,516609.10,17712.98,498896.12
2,Balanced solution-oriented Local Area Network,458227.42,84643.10,373584.32
3,Distributed real-time methodology,89958.73,14589.75,75368.98
4,Front-line executive infrastructure,47511.35,39291.90,8219.45


¿Qué canal de marketing se utiliza con mayor frecuencia y cuál genera mejor ROI?

In [11]:
# Calculate type frequency
type_counts = df['type'].value_counts()

# Calculate average ROI by type
roi_by_type = df.groupby('type')['roi_numeric'].mean()

# Create interactive subplots
fig = make_subplots(rows=1, cols=2, 
                   subplot_titles=['Frecuencia de uso de tipos de campaña', 
                                  'ROI promedio por tipo de campaña'])

# Plot type frequency
fig.add_trace(
    go.Bar(
        x=type_counts.index, 
        y=type_counts.values,
        text=type_counts.values,
        textposition='auto',
        marker_color=pastel_colors,
        hovertemplate='Tipo: %{x}<br>Frecuencia: %{y}<extra></extra>'
    ),
    row=1, col=1
)

# Plot average ROI by type
fig.add_trace(
    go.Bar(
        x=roi_by_type.index, 
        y=roi_by_type.values,
        text=[f'{val:.2%}' for val in roi_by_type.values],
        textposition='auto',
        marker_color=pastel_colors,
        hovertemplate='Tipo: %{x}<br>ROI: %{text}<extra></extra>'
    ),
    row=1, col=2
)

# Update layout
fig.update_layout(
    height=600, 
    width=1000,
    showlegend=False,
    title_text='Análisis de Tipos de Campaña de Marketing',
    template='simple_white'
)

# Add axis labels
fig.update_yaxes(title_text="Frecuencia", row=1, col=1)
fig.update_yaxes(title_text="ROI promedio", row=1, col=2)
fig.update_xaxes(title_text="Tipo de campaña", row=1, col=1)
fig.update_xaxes(title_text="Tipo de campaña", row=1, col=2)

# Show the figure
fig.show()


¿Qué tipo de campaña genera más ingresos en promedio y cuál tiene mejor conversión?

In [6]:
# Group by type and calculate metrics
avg_revenue_by_type = df.groupby('type')['revenue'].mean().sort_values(ascending=False)
avg_conversion_by_type = df.groupby('type')['conversion_rate'].mean().sort_values(ascending=False)

# Create interactive subplots
fig = make_subplots(rows=1, cols=2, 
                   subplot_titles=['Ingreso promedio por tipo de campaña', 
                                  'Tasa de conversión promedio por tipo de campaña'])

# Plot average revenue by campaign type
fig.add_trace(
    go.Bar(
        x=avg_revenue_by_type.index, 
        y=avg_revenue_by_type.values,
        text=[f'${val:,.2f}' for val in avg_revenue_by_type.values],
        textposition='auto',
        marker_color=pastel_colors[:len(avg_revenue_by_type)],
        hovertemplate='Tipo: %{x}<br>Ingreso promedio: %{text}<extra></extra>'
    ),
    row=1, col=1
)

# Plot average conversion rate by campaign type
fig.add_trace(
    go.Bar(
        x=avg_conversion_by_type.index, 
        y=avg_conversion_by_type.values,
        text=[f'{val:.2%}' for val in avg_conversion_by_type.values],
        textposition='auto',
        marker_color=pastel_colors[:len(avg_conversion_by_type)],
        hovertemplate='Tipo: %{x}<br>Tasa de conversión: %{text}<extra></extra>'
    ),
    row=1, col=2
)

# Update layout
fig.update_layout(
    height=600, 
    width=1000,
    showlegend=False,
    title_text='Análisis de Tipos de Campaña',
    template='simple_white'
)

# Add axis labels
fig.update_yaxes(title_text="Ingreso promedio ($)", row=1, col=1)
fig.update_yaxes(title_text="Tasa de conversión", row=1, col=2)
fig.update_xaxes(title_text="Tipo de campaña", row=1, col=1)
fig.update_xaxes(title_text="Tipo de campaña", row=1, col=2)

# Show the figure
fig.show()

In [7]:
from plotly.subplots import make_subplots
import numpy as np
from scipy import stats

# Create a figure with multiple subplots to analyze ROI distribution and factors
import plotly.express as px
import plotly.graph_objects as go

# Create a 2x2 subplot figure
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Distribución del ROI',
        'ROI por canal de marketing', 
        'ROI vs. Tasa de conversión',
        'ROI por tipo de campaña'
    ),
    specs=[[{"type": "histogram"}, {"type": "box"}],
           [{"type": "scatter"}, {"type": "box"}]]
)

# 1. ROI Distribution (histogram)
fig.add_trace(
    go.Histogram(
        x=df['roi_numeric'], 
        nbinsx=50,
        marker_color='rgba(255, 182, 193, 0.7)',
        hovertemplate='ROI: %{x:.2f}<br>Frecuencia: %{y}<extra></extra>'
    ),
    row=1, col=1
)

# 2. ROI by Channel (boxplot)
for i, channel in enumerate(df['channel'].unique()):
    fig.add_trace(
        go.Box(
            y=df[df['channel'] == channel]['roi_numeric'],
            name=channel,
            marker_color=pastel_colors[i % len(pastel_colors)],
            hovertemplate='Canal: %{y:.2f}<extra></extra>'
        ),
        row=1, col=2
    )

# 3. ROI vs Conversion Rate (scatter)
fig.add_trace(
    go.Scatter(
        x=df['conversion_rate'],
        y=df['roi_numeric'],
        mode='markers',
        marker=dict(
            size=8,
            color='rgba(173, 216, 230, 0.7)',
            line=dict(width=1, color='rgba(173, 216, 230, 1.0)')
        ),
        hovertemplate='Tasa de conversión: %{x:.2f}<br>ROI: %{y:.2f}<extra></extra>'
    ),
    row=2, col=1
)

# 4. ROI by Campaign Type (boxplot)
for i, campaign_type in enumerate(df['type'].unique()):
    fig.add_trace(
        go.Box(
            y=df[df['type'] == campaign_type]['roi_numeric'],
            name=campaign_type,
            marker_color=pastel_colors[i % len(pastel_colors)],
            hovertemplate='Tipo: %{y:.2f}<extra></extra>'
        ),
        row=2, col=2
    )

# Calculate correlation between ROI and conversion rate
roi_conv_corr = np.corrcoef(df['roi_numeric'], df['conversion_rate'])[0, 1]

# Update layout
fig.update_layout(
    height=800, 
    width=1000,
    title_text='Análisis de ROI y Factores Asociados',
    template='simple_white',
    showlegend=False,
    annotations=[
        dict(
            x=0.5,
            y=-0.1,
            xref="x2 domain",
            yref="y2 domain",
            text=f"Correlación entre ROI y tasa de conversión: {roi_conv_corr:.2f}",
            showarrow=False
        )
    ]
)

# Update axes
fig.update_xaxes(title_text="ROI", row=1, col=1)
fig.update_yaxes(title_text="Frecuencia", row=1, col=1)
fig.update_xaxes(title_text="Canal", row=1, col=2)
fig.update_yaxes(title_text="ROI", row=1, col=2)
fig.update_xaxes(title_text="Tasa de conversión", row=2, col=1)
fig.update_yaxes(title_text="ROI", row=2, col=1)
fig.update_xaxes(title_text="Tipo de campaña", row=2, col=2)
fig.update_yaxes(title_text="ROI", row=2, col=2)

fig.show()

In [8]:
# Group data by audience type
b2b_data = df[df['target_audience'] == 'B2B']['conversion_rate']
b2c_data = df[df['target_audience'] == 'B2C']['conversion_rate']

# Create visualizations to compare conversion rates
fig = make_subplots(rows=1, cols=2,
                   subplot_titles=['Distribución de tasas de conversión', 
                                  'Comparación de tasas de conversión'])

# Distribution plot (histograms)
fig.add_trace(
    go.Histogram(x=b2b_data, name='B2B', marker_color='rgba(173, 216, 230, 0.7)',
                opacity=0.7, nbinsx=20),
    row=1, col=1
)
fig.add_trace(
    go.Histogram(x=b2c_data, name='B2C', marker_color='rgba(255, 182, 193, 0.7)',
                opacity=0.7, nbinsx=20),
    row=1, col=1
)

# Boxplot comparison
fig.add_trace(
    go.Box(y=b2b_data, name='B2B', marker_color='rgba(173, 216, 230, 0.7)'),
    row=1, col=2
)
fig.add_trace(
    go.Box(y=b2c_data, name='B2C', marker_color='rgba(255, 182, 193, 0.7)'),
    row=1, col=2
)

# Update layout
fig.update_layout(
    height=500, 
    width=1000,
    title_text='Análisis de Tasas de Conversión: B2B vs B2C',
    template='simple_white',
    barmode='overlay'
)

# Update axes
fig.update_xaxes(title_text="Tasa de conversión", row=1, col=1)
fig.update_yaxes(title_text="Frecuencia", row=1, col=1)
fig.update_xaxes(title_text="Audiencia", row=1, col=2)
fig.update_yaxes(title_text="Tasa de conversión", row=1, col=2)

# Show plot
fig.show()

# Perform statistical test
from scipy import stats as scipy_stats
t_stat, p_value = scipy_stats.ttest_ind(b2b_data, b2c_data, equal_var=False)

# Calculate mean conversion rates
mean_b2b = b2b_data.mean()
mean_b2c = b2c_data.mean()
diff_percent = abs(mean_b2b - mean_b2c) / ((mean_b2b + mean_b2c) / 2) * 100

# Display results
print(f"Tasa promedio de conversión B2B: {mean_b2b:.2%}")
print(f"Tasa promedio de conversión B2C: {mean_b2c:.2%}")
print(f"Diferencia porcentual: {diff_percent:.2f}%")
print(f"Resultado de la prueba t: t = {t_stat:.4f}, p-valor = {p_value:.4f}")
print(f"Conclusión: {'Hay diferencias significativas' if p_value < 0.05 else 'No hay diferencias significativas'} entre las tasas de conversión de B2B y B2C (α=0.05)")

Tasa promedio de conversión B2B: 55.13%
Tasa promedio de conversión B2C: 53.60%
Diferencia porcentual: 2.80%
Resultado de la prueba t: t = 0.9036, p-valor = 0.3664
Conclusión: No hay diferencias significativas entre las tasas de conversión de B2B y B2C (α=0.05)


In [9]:
# Ordena las campañas por beneficio neto de mayor a menor
top_campaigns = df.sort_values(by='net_profit', ascending=False).head(10)

# Crea una tabla con las 10 mejores campañas
fig = go.Figure(data=[go.Table(
    header=dict(
        values=['Ranking', 'Campaña', 'Beneficio Neto', 'ROI', 'Canal', 'Tipo', 'Audiencia', 'Tasa Conv.'],
        fill_color='rgba(173, 216, 230, 0.7)',
        align='left'
    ),
    cells=dict(
        values=[
            list(range(1, len(top_campaigns) + 1)),
            top_campaigns['campaign_name'],
            ['${:,.2f}'.format(val) for val in top_campaigns['net_profit']],
            top_campaigns['roi'],
            top_campaigns['channel'],
            top_campaigns['type'],
            top_campaigns['target_audience'],
            ['{:.2%}'.format(val) for val in top_campaigns['conversion_rate']]
        ],
        fill_color=[['rgba(255, 255, 255, 0.8)'] * len(top_campaigns)],
        align='left')
)])

fig.update_layout(
    title='Top 10 Campañas por Beneficio Neto',
    height=400,
    width=1000
)
fig.show()

# Analiza características comunes de las campañas más exitosas
fig2 = make_subplots(
    rows=2, cols=2,
    specs=[
        [{'type': 'domain'}, {'type': 'domain'}],  # Row 1: pie charts
        [{'type': 'domain'}, {'type': 'xy'}]       # Row 2: pie chart and scatter plot
    ],
    subplot_titles=(
        'Canales en Top 10 Campañas', 
        'Tipos de Campaña en Top 10',
        'Audiencia Objetivo en Top 10', 
        'Tasa de Conversión vs. Beneficio Neto'
    )
)

# Distribución de canales
channel_counts = top_campaigns['channel'].value_counts()
fig2.add_trace(
    go.Pie(
        labels=channel_counts.index, 
        values=channel_counts.values,
        textinfo='percent+label',
        marker=dict(colors=pastel_colors),
        hovertemplate='Canal: %{label}<br>Cantidad: %{value}<br>Porcentaje: %{percent}<extra></extra>'
    ),
    row=1, col=1
)

# Distribución de tipos
type_counts = top_campaigns['type'].value_counts()
fig2.add_trace(
    go.Pie(
        labels=type_counts.index, 
        values=type_counts.values,
        textinfo='percent+label',
        marker=dict(colors=pastel_colors),
        hovertemplate='Tipo: %{label}<br>Cantidad: %{value}<br>Porcentaje: %{percent}<extra></extra>'
    ),
    row=1, col=2
)

# Distribución de audiencia
audience_counts = top_campaigns['target_audience'].value_counts()
fig2.add_trace(
    go.Pie(
        labels=audience_counts.index, 
        values=audience_counts.values,
        textinfo='percent+label',
        marker=dict(colors=pastel_colors[:len(audience_counts)]),
        hovertemplate='Audiencia: %{label}<br>Cantidad: %{value}<br>Porcentaje: %{percent}<extra></extra>'
    ),
    row=2, col=1
)

# Tasa de conversión vs beneficio neto
fig2.add_trace(
    go.Scatter(
        x=top_campaigns['conversion_rate'],
        y=top_campaigns['net_profit'],
        mode='markers+text',
        text=[f"{i+1}" for i in range(len(top_campaigns))],
        textposition="top center",
        marker=dict(
            size=12,
            color=[i for i in range(len(top_campaigns))],
            colorscale='Viridis',
            showscale=False
        ),
        hovertemplate='Ranking: %{text}<br>Tasa Conv.: %{x:.2%}<br>Beneficio Neto: $%{y:,.2f}<extra></extra>'
    ),
    row=2, col=2
)

# Actualiza el diseño
fig2.update_layout(
    height=800, 
    width=1000,
    title_text='Análisis de Características de las 10 Campañas Más Exitosas',
    template='simple_white'
)

# Etiquetas de los ejes
fig2.update_xaxes(title_text="Tasa de Conversión", row=2, col=2)
fig2.update_yaxes(title_text="Beneficio Neto ($)", row=2, col=2)

# Muestra la figura
fig2.show()

# Análisis estadístico de las campañas top vs resto
print("Estadísticas comparativas - Top 10 vs Resto de campañas:")
metrics = ['budget', 'conversion_rate', 'revenue', 'campaign_duration']
for metric in metrics:
    top_mean = top_campaigns[metric].mean()
    rest_mean = df[~df['campaign_name'].isin(top_campaigns['campaign_name'])][metric].mean()
    diff_percent = (top_mean - rest_mean) / rest_mean * 100
    
    print(f"{metric.capitalize()}: Top 10 = {top_mean:.2f} vs Resto = {rest_mean:.2f} (Dif: {diff_percent:+.2f}%)")

Estadísticas comparativas - Top 10 vs Resto de campañas:
Budget: Top 10 = 14938.40 vs Resto = 49808.88 (Dif: -70.01%)
Conversion_rate: Top 10 = 0.54 vs Resto = 0.54 (Dif: -0.66%)
Revenue: Top 10 = 986461.87 vs Resto = 509849.73 (Dif: +93.48%)
Campaign_duration: Top 10 = 425.80 vs Resto = 365.60 (Dif: +16.46%)


In [10]:
from statsmodels.nonparametric.smoothers_lowess import lowess

# Analyze the relationship between budget and revenue/ROI

# Calculate correlations
correlations = {
    'Budget vs Revenue': df['budget'].corr(df['revenue']),
    'Budget vs ROI': df['budget'].corr(df['roi_numeric']),
    'Budget vs Net Profit': df['budget'].corr(df['net_profit']),
    'Budget vs Conversion Rate': df['budget'].corr(df['conversion_rate'])
}

# Create subplots for analysis with improved spacing
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=[
        'Relación entre Presupuesto e Ingresos',
        'Eficiencia de Inversión por Rangos',
        'Presupuesto vs ROI',
        'Análisis por Canal'
    ],
    horizontal_spacing=0.12,  # Increase horizontal spacing
    vertical_spacing=0.15     # Increase vertical spacing
)

# 1. Scatter plot with trend lines (Budget vs Revenue)
# Add linear trend
df_sorted = df.sort_values('budget')
x = df_sorted['budget']
y = df_sorted['revenue']

# Add linear trendline
model_lin = sm.OLS(y, sm.add_constant(x)).fit()
y_pred_lin = model_lin.predict(sm.add_constant(x))

# Add polynomial trendline (degree 2)
model_poly = sm.OLS(y, sm.add_constant(np.column_stack((x, x**2)))).fit()
y_pred_poly = model_poly.predict(sm.add_constant(np.column_stack((x, x**2))))

# Add data points with improved visibility
fig.add_trace(
    go.Scatter(
        x=df['budget'], 
        y=df['revenue'],
        mode='markers',
        marker=dict(
            color=df['roi_numeric'],
            colorscale='Viridis',
            colorbar=dict(
                title='ROI',
                thickness=15,
                len=0.3,
                y=0.8,
                yanchor='top',
                outlinewidth=1
            ),
            size=8,
            opacity=0.7
        ),
        name='Campañas',
        hovertemplate='Presupuesto: $%{x:,.2f}<br>Ingresos: $%{y:,.2f}<br>ROI: %{marker.color:.2f}<extra></extra>'
    ),
    row=1, col=1
)

# Add linear trendline
fig.add_trace(
    go.Scatter(
        x=x, 
        y=y_pred_lin,
        mode='lines',
        line=dict(color='rgba(255, 0, 0, 0.7)', width=2),
        name=f'Lineal (R²={model_lin.rsquared:.3f})'
    ),
    row=1, col=1
)

# Add polynomial trendline
fig.add_trace(
    go.Scatter(
        x=x, 
        y=y_pred_poly,
        mode='lines',
        line=dict(color='rgba(0, 128, 0, 0.7)', width=2, dash='dash'),
        name=f'Polinomial (R²={model_poly.rsquared:.3f})'
    ),
    row=1, col=1
)

# 2. Efficiency analysis by budget ranges - improved bar chart
budget_bins = pd.qcut(df['budget'], 5)
efficiency_by_bin = df.groupby(budget_bins).apply(
    lambda x: pd.Series({
        'avg_budget': x['budget'].mean(),
        'avg_revenue': x['revenue'].mean(),
        'roi': x['roi_numeric'].mean(),
        'efficiency': x['revenue'].sum() / x['budget'].sum(),
        'count': len(x)
    })
)

# Format range labels to be shorter and clearer
range_labels = [f'${b.left/1000:.0f}k-${b.right/1000:.0f}k' for b in efficiency_by_bin.index]

fig.add_trace(
    go.Bar(
        x=range_labels,
        y=efficiency_by_bin['efficiency'],
        marker_color='rgba(173, 216, 230, 0.8)',
        text=[f'{e:.1f}x' for e in efficiency_by_bin['efficiency']],
        textposition='auto',
        textfont=dict(size=12, color='black'),
        hovertemplate='<b>Rango:</b> %{x}<br><b>Retorno por $ invertido:</b> %{y:.2f}x<br><b>ROI medio:</b> %{text}<extra></extra>'
    ),
    row=1, col=2
)

# 3. Budget vs ROI - improved visualization
fig.add_trace(
    go.Scatter(
        x=df['budget'],
        y=df['roi_numeric'],
        mode='markers',
        marker=dict(
            color=df['revenue'],
            colorscale='Plasma',
            colorbar=dict(
                title='Ingresos ($)',
                thickness=15,
                len=0.3,
                y=0.25,
                yanchor='bottom',
                outlinewidth=1
            ),
            size=8,
            opacity=0.7
        ),
        hovertemplate='<b>Presupuesto:</b> $%{x:,.2f}<br><b>ROI:</b> %{y:.2f}<br><b>Ingresos:</b> $%{marker.color:,.2f}<extra></extra>'
    ),
    row=2, col=1
)

# Add locally weighted regression (LOWESS) to see trends
lowess_result = lowess(df['roi_numeric'], df['budget'], frac=0.3)

fig.add_trace(
    go.Scatter(
        x=lowess_result[:, 0],
        y=lowess_result[:, 1],
        mode='lines',
        line=dict(color='rgba(255, 0, 0, 0.7)', width=3),
        name='Tendencia LOWESS'
    ),
    row=2, col=1
)

# 4. Analysis by channel - improved colors and legend
for i, channel in enumerate(df['channel'].unique()):
    channel_data = df[df['channel'] == channel]
    
    fig.add_trace(
        go.Scatter(
            x=channel_data['budget'],
            y=channel_data['revenue'],
            mode='markers',
            marker=dict(
                color=pastel_colors[i % len(pastel_colors)],
                size=8
            ),
            name=channel,
            legendgroup=channel,
            hovertemplate=f'<b>Canal:</b> {channel}<br><b>Presupuesto:</b> $%{{x:,.2f}}<br><b>Ingresos:</b> $%{{y:,.2f}}<extra></extra>'
        ),
        row=2, col=2
    )

# Highlight the optimal investment range
optimal_range = efficiency_by_bin.iloc[efficiency_by_bin['efficiency'].argmax()]
optimal_index = efficiency_by_bin['efficiency'].argmax()
optimal_label = range_labels[optimal_index]

# Update layout with better overall appearance
fig.update_layout(
    height=900,  # Increase height
    width=1100,  # Increase width
    title={
        'text': 'Análisis de la Relación entre Presupuesto e Ingresos',
        'y': 0.98,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(size=20)
    },
    template='simple_white',
    legend=dict(
        orientation="h", 
        yanchor="bottom", 
        y=1.05, 
        xanchor="center", 
        x=0.5,
        font=dict(size=12),
        borderwidth=1,
        bordercolor="LightGrey"
    ),
    margin=dict(l=80, r=80, t=120, b=80)  # Add more margin
)

# Update axes with improved formatting
fig.update_xaxes(title_text='Presupuesto ($)', title_font=dict(size=14), tickfont=dict(size=12), row=1, col=1)
fig.update_yaxes(title_text='Ingresos ($)', title_font=dict(size=14), tickfont=dict(size=12), row=1, col=1)

fig.update_xaxes(title_text='Rango de presupuesto', title_font=dict(size=14), tickfont=dict(size=11), row=1, col=2)
fig.update_yaxes(title_text='Retorno por $ invertido', title_font=dict(size=14), tickfont=dict(size=12), row=1, col=2)

fig.update_xaxes(title_text='Presupuesto ($)', title_font=dict(size=14), tickfont=dict(size=12), row=2, col=1)
fig.update_yaxes(title_text='ROI', title_font=dict(size=14), tickfont=dict(size=12), row=2, col=1)

fig.update_xaxes(title_text='Presupuesto ($)', title_font=dict(size=14), tickfont=dict(size=12), row=2, col=2)
fig.update_yaxes(title_text='Ingresos ($)', title_font=dict(size=14), tickfont=dict(size=12), row=2, col=2)

# Highlight optimal investment range in the second chart
fig.update_traces(
    marker_color=['rgba(255, 165, 0, 0.8)' if i == optimal_index else 'rgba(173, 216, 230, 0.8)' for i in range(len(efficiency_by_bin))],
    selector=dict(type='bar')
)

# Add annotation for optimal range
fig.add_annotation(
    x=optimal_label,
    y=optimal_range['efficiency'] + 2,
    text=f"Punto óptimo<br>{optimal_label}<br>Eficiencia: {optimal_range['efficiency']:.1f}x",
    showarrow=True,
    arrowhead=1,
    arrowsize=1,
    arrowwidth=2,
    arrowcolor='red',
    font=dict(size=12, color='red'),
    bgcolor='rgba(255, 255, 255, 0.8)',
    bordercolor='red',
    borderwidth=1,
    borderpad=4,
    row=1, col=2
)

# Show the plot and summary analysis
fig.show()

# Print key findings
print(f"Correlaciones:")
for k, v in correlations.items():
    print(f"- {k}: {v:.3f}")

print("\nAnálisis de eficiencia por rango de presupuesto:")
for idx, row in efficiency_by_bin.iterrows():
    print(f"- Rango ${idx.left:.0f}-${idx.right:.0f}: Eficiencia: {row['efficiency']:.2f}x, ROI medio: {row['roi']:.2f}, Campañas: {row['count']}")

print(f"\nPunto óptimo de inversión: Rango ${optimal_data.name.left:.0f}-${optimal_data.name.right:.0f}")
print(f"- Presupuesto promedio: ${optimal_data['avg_budget']:.2f}")
print(f"- Retorno por $ invertido: {optimal_data['efficiency']:.2f}x")
print(f"- ROI promedio: {optimal_data['roi']:.2f}")

C:\Users\Andrei.Baidurov\AppData\Local\Temp\ipykernel_18376\474956458.py:92: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



Correlaciones:
- Budget vs Revenue: -0.014
- Budget vs ROI: -0.431
- Budget vs Net Profit: -0.113
- Budget vs Conversion Rate: -0.015

Análisis de eficiencia por rango de presupuesto:
- Rango $1053-$20387: Eficiencia: 50.54x, ROI medio: 86.01, Campañas: 202.0
- Rango $20387-$38877: Eficiencia: 16.84x, ROI medio: 16.49, Campañas: 202.0
- Rango $38877-$58916: Eficiencia: 11.46x, ROI medio: 10.60, Campañas: 202.0
- Rango $58916-$80313: Eficiencia: 6.99x, ROI medio: 6.08, Campañas: 202.0
- Rango $80313-$99957: Eficiencia: 5.76x, ROI medio: 4.79, Campañas: 202.0


NameError: name 'optimal_data' is not defined

In [ ]:
# Identify high-performance campaigns using multiple criteria
import plotly.figure_factory as ff

# Define performance thresholds (75th percentile for each metric)
roi_threshold = df['roi_numeric'].quantile(0.75)
conversion_threshold = df['conversion_rate'].quantile(0.75)
efficiency_threshold = (df['revenue'] / df['budget']).quantile(0.75)

# Create filter masks for each criterion
high_roi = df['roi_numeric'] >= roi_threshold
high_conversion = df['conversion_rate'] >= conversion_threshold
high_efficiency = (df['revenue'] / df['budget']) >= efficiency_threshold

# Identify campaigns that meet multiple criteria
# All three criteria
elite_campaigns = df[high_roi & high_conversion & high_efficiency].copy()
# At least two criteria
strong_campaigns = df[(high_roi & high_conversion) | (high_roi & high_efficiency) | 
                     (high_conversion & high_efficiency)].copy()
strong_campaigns = strong_campaigns[~strong_campaigns.index.isin(elite_campaigns.index)]

# Add performance category
elite_campaigns['performance_category'] = 'Elite (3 criterios)'
strong_campaigns['performance_category'] = 'Fuerte (2 criterios)'

# Combine the filtered campaigns
high_performers = pd.concat([elite_campaigns, strong_campaigns])

# Calculate proportion of successful campaigns by channel and type
success_by_channel = high_performers['channel'].value_counts() / df['channel'].value_counts() * 100
success_by_type = high_performers['type'].value_counts() / df['type'].value_counts() * 100

# Create a scatter plot with quadrants
fig = make_subplots(
    rows=2, cols=2,
    column_widths=[0.65, 0.35],
    row_heights=[0.6, 0.4],
    specs=[
        [{"type": "scatter"}, {"type": "bar"}],
        [{"type": "table", "colspan": 2}, None]
    ],
    subplot_titles=("Mapa de Rendimiento de Campañas", 
                   "Tasa de Éxito por Canal",
                   "Campañas de Alto Rendimiento")
)

# Add main scatter plot (ROI vs Conversion Rate, size = Budget, color = Efficiency)
sizes = (high_performers['budget'] / high_performers['budget'].max() * 50) + 10
fig.add_trace(
    go.Scatter(
        x=high_performers['roi_numeric'],
        y=high_performers['conversion_rate'],
        mode='markers',
        marker=dict(
            size=sizes,
            color=high_performers['revenue'] / high_performers['budget'],
            colorscale='Viridis',
            colorbar=dict(title="Eficiencia"),
            showscale=True,
            line=dict(width=1, color='white')
        ),
        text=high_performers['campaign_name'],
        customdata=np.stack((
            high_performers['performance_category'],
            high_performers['channel'],
            high_performers['type'],
            high_performers['budget'],
            high_performers['revenue'],
            high_performers['campaign_duration']
        ), axis=1),
        hovertemplate='<b>%{text}</b><br>' +
                      'Categoría: %{customdata[0]}<br>' +
                      'Canal: %{customdata[1]}<br>' +
                      'Tipo: %{customdata[2]}<br>' +
                      'ROI: %{x:.2f}<br>' +
                      'Conv.: %{y:.2%}<br>' +
                      'Presupuesto: $%{customdata[3]:,.2f}<br>' +
                      'Ingresos: $%{customdata[4]:,.2f}<br>' +
                      'Duración: %{customdata[5]} días',
    ),
    row=1, col=1
)

# Add threshold lines
fig.add_shape(
    type="line", line=dict(dash="dash", color="red"),
    x0=roi_threshold, y0=0, x1=roi_threshold, y1=1.6,
    row=1, col=1
)
fig.add_shape(
    type="line", line=dict(dash="dash", color="red"),
    x0=0, y0=conversion_threshold, x1=900, y1=conversion_threshold,
    row=1, col=1
)

# Add annotations for quadrants
fig.add_annotation(
    x=roi_threshold*1.5, y=conversion_threshold*1.5,
    text="Alto ROI + Alta Conversión",
    showarrow=False, font=dict(size=10),
    bgcolor="rgba(255, 255, 255, 0.8)",
    row=1, col=1
)

# Add success rate by channel bar chart
fig.add_trace(
    go.Bar(
        y=success_by_channel.index,
        x=success_by_channel.values,
        orientation='h',
        marker_color=pastel_colors[:len(success_by_channel)],
        text=[f'{x:.1f}%' for x in success_by_channel.values],
        textposition='auto',
        hovertemplate='<b>%{y}</b><br>Tasa de éxito: %{x:.1f}%',
    ),
    row=1, col=2
)

# Create a table with top performers
top_high_performers = high_performers.nlargest(10, 'revenue')
table_data = [
    top_high_performers['campaign_name'],
    top_high_performers['performance_category'],
    top_high_performers['channel'],
    top_high_performers['type'],
    [f'{x:.2f}' for x in top_high_performers['roi_numeric']],
    [f'{x:.2%}' for x in top_high_performers['conversion_rate']],
    [f'${x:,.0f}' for x in top_high_performers['revenue']],
]

fig.add_trace(
    go.Table(
        header=dict(
            values=['Campaña', 'Categoría', 'Canal', 'Tipo', 'ROI', 'Conv', 'Ingresos'],
            fill_color='rgba(173, 216, 230, 0.7)',
            align='left',
            font=dict(size=12)
        ),
        cells=dict(
            values=table_data,
            fill_color='white',
            align='left',
            font=dict(size=11),
            height=30
        )
    ),
    row=2, col=1
)

# Update layout
fig.update_layout(
    height=900,
    width=1100,
    title_text="Análisis de Campañas de Alto Rendimiento Multidimensional",
    showlegend=False,
    template='simple_white',
)

# Update axes
fig.update_xaxes(title_text="ROI", row=1, col=1)
fig.update_yaxes(title_text="Tasa de Conversión", row=1, col=1)
fig.update_xaxes(title_text="Tasa de Éxito (%)", row=1, col=2)

# Show plot
fig.show()

# Print summary statistics
print(f"Campañas de rendimiento élite (3 criterios): {len(elite_campaigns)}")
print(f"Campañas de rendimiento fuerte (2 criterios): {len(strong_campaigns)}")

# Calculate average metrics by performance category
performance_metrics = high_performers.groupby('performance_category').agg({
    'roi_numeric': 'mean',
    'conversion_rate': 'mean',
    'revenue': 'mean',
    'budget': 'mean',
    'campaign_duration': 'mean',
    'revenue': lambda x: x.sum() / high_performers['budget'].sum()
}).rename(columns={'revenue': 'overall_efficiency'})

print("\nMétricas promedio por categoría:")
print(performance_metrics)

# Identify common characteristics
print("\nCaracterísticas comunes de campañas de alto rendimiento:")
print(f"- Canales dominantes: {', '.join(success_by_channel.nlargest(2).index.tolist())}")
print(f"- Tipos dominantes: {', '.join(success_by_type.nlargest(2).index.tolist())}")

Campañas de rendimiento élite (3 criterios): 72
Campañas de rendimiento fuerte (2 criterios): 181

Métricas promedio por categoría:
                      roi_numeric  conversion_rate  overall_efficiency  \
performance_category                                                     
Elite (3 criterios)     75.882917         0.863056           11.867294   
Fuerte (2 criterios)    77.324420         0.424586           29.245735   

                            budget  campaign_duration  
performance_category                                   
Elite (3 criterios)   16782.762500         358.402778  
Fuerte (2 criterios)  16415.448398         375.933702  

Características comunes de campañas de alto rendimiento:
- Canales dominantes: organic, promotion
- Tipos dominantes: email, social media


In [ ]:
import pandas as pd
from datetime import datetime
from plotly.subplots import make_subplots
import numpy as np
import calendar
from scipy import stats
from scipy.stats import f_oneway
import statsmodels.api as sm

# Convert date strings to datetime objects and extract time components
import plotly.graph_objects as go
import plotly.express as px

# Define pastel colors for charts
pastel_colors = ['rgba(255, 182, 193, 0.7)', 'rgba(173, 216, 230, 0.7)', 
                'rgba(144, 238, 144, 0.7)', 'rgba(255, 218, 185, 0.7)']

# 1. Process dates and extract time components
df['start_date'] = pd.to_datetime(df['start_date'])
df['end_date'] = pd.to_datetime(df['end_date'])

# Extract temporal components
df['start_year'] = df['start_date'].dt.year
df['start_month'] = df['start_date'].dt.month
df['start_quarter'] = df['start_date'].dt.quarter
df['month_name'] = df['start_date'].dt.strftime('%b')
df['campaign_length'] = (df['end_date'] - df['start_date']).dt.days

# Define seasons (Northern Hemisphere)
def get_season(month):
    if month in [12, 1, 2]:
        return 'Invierno'
    elif month in [3, 4, 5]:
        return 'Primavera'
    elif month in [6, 7, 8]:
        return 'Verano'
    else:  # month in [9, 10, 11]
        return 'Otoño'

df['season'] = df['start_month'].apply(get_season)

# Create subplots with different temporal analyses
fig = make_subplots(
    rows=3, cols=2,
    subplot_titles=(
        'Rendimiento por Temporada',
        'Tendencia de ROI a lo Largo del Tiempo',
        'Rendimiento Mensual (Heatmap)',
        'Tasa de Conversión por Mes y Canal',
        'Ingresos Medios por Trimestre',
        'Duración de Campaña vs. Rendimiento'
    ),
    specs=[
        [{"type": "xy", "secondary_y": True}, {"type": "xy", "secondary_y": True}],
        [{"type": "heatmap"}, {"type": "bar"}],
        [{"type": "xy", "secondary_y": True}, {"type": "scatter"}]
    ],
    vertical_spacing=0.12,
    horizontal_spacing=0.1
)

# 1. Performance by Season (Bar chart)
seasonal_perf = df.groupby('season').agg({
    'roi_numeric': 'mean',
    'conversion_rate': 'mean',
    'revenue': 'mean',
    'net_profit': 'mean'
}).reset_index()

# Custom season order
season_order = ['Primavera', 'Verano', 'Otoño', 'Invierno']
seasonal_perf['season'] = pd.Categorical(seasonal_perf['season'], categories=season_order, ordered=True)
seasonal_perf = seasonal_perf.sort_values('season')

# Multiple metrics in one chart using secondary y-axis
fig.add_trace(
    go.Bar(
        x=seasonal_perf['season'],
        y=seasonal_perf['roi_numeric'],
        name='ROI',
        marker_color='rgba(173, 216, 230, 0.7)',
        text=[f"{x:.1f}" for x in seasonal_perf['roi_numeric']],
        textposition='auto'
    ),
    row=1, col=1, secondary_y=False
)

fig.add_trace(
    go.Scatter(
        x=seasonal_perf['season'],
        y=seasonal_perf['conversion_rate'],
        name='Tasa de Conversión',
        mode='lines+markers',
        line=dict(color='rgba(255, 0, 0, 0.7)', width=2),
        marker=dict(size=10)
    ),
    row=1, col=1, secondary_y=True
)

# 2. ROI Trend Over Time (Line plot)
# Group by year and month for time series
time_trend = df.groupby(['start_year', 'start_month']).agg({
    'roi_numeric': 'mean',
    'conversion_rate': 'mean',
    'revenue': 'mean',
    'campaign_name': 'count'
}).reset_index()

# Create a proper date column for continuous time series
time_trend['date'] = pd.to_datetime({
    'year': time_trend['start_year'],
    'month': time_trend['start_month'],
    'day': 1
})
time_trend = time_trend.sort_values('date')

fig.add_trace(
    go.Scatter(
        x=time_trend['date'],
        y=time_trend['roi_numeric'],
        mode='lines+markers',
        name='ROI Promedio',
        line=dict(color='rgba(0, 128, 0, 0.7)', width=2),
        marker=dict(size=8)
    ),
    row=1, col=2, secondary_y=False
)

fig.add_trace(
    go.Scatter(
        x=time_trend['date'],
        y=time_trend['revenue'],
        mode='lines',
        name='Ingresos',
        line=dict(color='rgba(255, 165, 0, 0.5)', width=2, dash='dash')
    ),
    row=1, col=2, secondary_y=True
)

fig.add_trace(
    go.Scatter(
        x=time_trend['date'],
        y=time_trend['campaign_name'],
        mode='lines',
        name='Número de Campañas',
        line=dict(color='rgba(128, 0, 128, 0.5)', width=2, dash='dot')
    ),
    row=1, col=2, secondary_y=True
)

# 3. Monthly Performance Heatmap
# Create a cross-tabulation of month vs. type with ROI values
monthly_type_roi = df.pivot_table(
    values='roi_numeric', 
    index='month_name', 
    columns='type',
    aggfunc='mean'
)

# Reorder the months correctly
month_order = [calendar.month_abbr[i] for i in range(1, 13)]
monthly_type_roi = monthly_type_roi.reindex(month_order)

fig.add_trace(
    go.Heatmap(
        z=monthly_type_roi.values,
        x=monthly_type_roi.columns,
        y=monthly_type_roi.index,
        colorscale='Viridis',
        colorbar=dict(
            title='ROI Promedio',
            len=0.3,
            y=0.45,
            yanchor='middle'
        ),
        hovertemplate='Tipo: %{x}<br>Mes: %{y}<br>ROI: %{z:.2f}<extra></extra>'
    ),
    row=2, col=1
)

# 4. Conversion Rate by Month and Channel
monthly_channel_conv = df.groupby(['month_name', 'channel']).agg({
    'conversion_rate': 'mean'
}).reset_index()

# Sort by custom month order
month_dict = {m: i for i, m in enumerate(month_order)}
monthly_channel_conv['month_num'] = monthly_channel_conv['month_name'].map(month_dict)
monthly_channel_conv = monthly_channel_conv.sort_values('month_num')

for i, channel in enumerate(df['channel'].unique()):
    channel_data = monthly_channel_conv[monthly_channel_conv['channel'] == channel]
    
    fig.add_trace(
        go.Bar(
            x=channel_data['month_name'],
            y=channel_data['conversion_rate'],
            name=channel,
            marker_color=pastel_colors[i % len(pastel_colors)],
            text=[f"{x:.1%}" for x in channel_data['conversion_rate']],
            textposition='auto'
        ),
        row=2, col=2
    )

# 5. Average Revenue by Quarter
quarterly_perf = df.groupby(['start_year', 'start_quarter']).agg({
    'revenue': 'mean', 
    'roi_numeric': 'mean',
    'campaign_name': 'count'
}).reset_index()
quarterly_perf['quarter_label'] = 'Q' + quarterly_perf['start_quarter'].astype(str) + ' ' + quarterly_perf['start_year'].astype(str)
quarterly_perf = quarterly_perf.sort_values(['start_year', 'start_quarter'])

fig.add_trace(
    go.Bar(
        x=quarterly_perf['quarter_label'],
        y=quarterly_perf['revenue'],
        marker_color='rgba(144, 238, 144, 0.7)',
        name='Ingresos Medios',
        text=[f"${x/1000:.0f}k" for x in quarterly_perf['revenue']],
        textposition='auto'
    ),
    row=3, col=1, secondary_y=False
)

fig.add_trace(
    go.Scatter(
        x=quarterly_perf['quarter_label'],
        y=quarterly_perf['roi_numeric'],
        mode='lines+markers',
        name='ROI',
        line=dict(color='rgba(255, 0, 0, 0.7)', width=2),
        marker=dict(size=8)
    ),
    row=3, col=1, secondary_y=True
)

# 6. Campaign Length vs Performance (Scatter plot with regression)
fig.add_trace(
    go.Scatter(
        x=df['campaign_length'],
        y=df['roi_numeric'],
        mode='markers',
        name='Campañas',
        marker=dict(
            size=8,
            color=df['conversion_rate'],
            colorscale='Viridis',
            colorbar=dict(
                title='Conv. Rate', 
                len=0.3, 
                y=0.15, 
                yanchor='middle'
            ),
            showscale=True
        )
    ),
    row=3, col=2
)

# Add trend line using lowess
lowess_result = sm.nonparametric.lowess(df['roi_numeric'], df['campaign_length'], frac=0.3)

fig.add_trace(
    go.Scatter(
        x=lowess_result[:, 0],
        y=lowess_result[:, 1],
        mode='lines',
        name='Tendencia',
        line=dict(color='red', width=2)
    ),
    row=3, col=2
)

# Update layout with better axis labels and secondary y-axes
fig.update_layout(
    height=1200, 
    width=1200,
    title={
        'text': 'Análisis Temporal de Campañas de Marketing',
        'y': 0.98,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    template='simple_white',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.5,
        itemsizing='constant'
    ),
    hovermode='closest',
    margin=dict(t=120, b=50, l=60, r=60),
    font=dict(size=10)
)

# Update axes with correct labels and formatting
fig.update_xaxes(title_text="Temporada", row=1, col=1)
fig.update_yaxes(title_text="ROI Promedio", row=1, col=1, secondary_y=False)
fig.update_yaxes(title_text="Tasa de Conversión", row=1, col=1, secondary_y=True)

fig.update_xaxes(title_text="Fecha", row=1, col=2)
fig.update_yaxes(title_text="ROI Promedio", row=1, col=2, secondary_y=False)
fig.update_yaxes(title_text="Ingresos/Nº de Campañas", row=1, col=2, secondary_y=True)

fig.update_xaxes(title_text="Tipo de Campaña", row=2, col=1)
fig.update_yaxes(title_text="Mes", row=2, col=1)

fig.update_xaxes(title_text="Mes", row=2, col=2)
fig.update_yaxes(title_text="Tasa de Conversión", row=2, col=2)

fig.update_xaxes(title_text="Trimestre", row=3, col=1, tickangle=45)
fig.update_yaxes(title_text="Ingresos Promedio", row=3, col=1, secondary_y=False)
fig.update_yaxes(title_text="ROI", row=3, col=1, secondary_y=True)

fig.update_xaxes(title_text="Duración de la Campaña (días)", row=3, col=2)
fig.update_yaxes(title_text="ROI", row=3, col=2)

# Show the figure
fig.show()

# Statistical Analysis section remains unchanged
